<a href="https://colab.research.google.com/github/Lenguist/insight-game-ai/blob/main/simple_sim_united.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Description moved to readme

# Simulation code (only know # of turns)


In [ ]:
from simulations import basic_simulation
from sellers import RandomSeller
from sellers import DescentArithmeticSeller
imp_list = [i / 10.0 for i in range(11)]
all_results = []

# params that are needed to initialize seller
value = 10
range_min = 11
range_max = 30
rounds = 10000
init_offer=range_max

# random agent
random_results = []
seller = RandomSeller(value=value, init_offer=init_offer)
for imp_incr in imp_list:
  result = basic_simulation(value,
                        range_min,
                        range_max,
                        imp_incr = imp_incr,
                        imp_init = 0,
                        rounds = rounds,
                        seller=seller)
  random_results.append(result)

# simple heuristic agent, consistently performs better than random
arithmetic_results = []
n = int((init_offer-value)/2)
seller = DescentArithmeticSeller(value=value, init_offer=init_offer, init_descent = n)
for imp_incr in imp_list:
  result = basic_simulation(value,
                        range_min,
                        range_max,
                        imp_incr = imp_incr,
                        imp_init = 0,
                        rounds = rounds,
                        seller=seller)
  arithmetic_results.append(result)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(imp_list, random_results, marker='o', label=f'RandomSeller')
plt.plot(imp_list, arithmetic_results, marker='o', label=f'DescentArithmeticSeller')
plt.title('Payoff vs. Imp_incr')
plt.xlabel('Impatience Incremenet')
plt.ylabel('Average Episode Payoff')
plt.grid()

plt.legend()  # This will create a legend using the labels specified in the plot functions

plt.show()

In [ ]:
from simulations import *
imp_list = [i / 10.0 for i in range(11)]
all_results = []

# params that are needed to initialize seller
value = 10
range_min = 11
range_max = 30
rounds = 10000
init_offer=range_max

# random agent
seller = RandomSeller(value=value, init_offer=init_offer)
random_results = random_imp_simulation(value,
                        range_min,
                        range_max,
                        rounds = rounds,
                        seller=seller)

# simple heuristic agent, consistently performs better than random
n = int((init_offer-value)/2)
seller = DescentArithmeticSeller(value=value, init_offer=init_offer, init_descent = n)
arithmetic_results = random_imp_simulation(value,
                        range_min,
                        range_max,
                        rounds = rounds,
                        seller=seller)

In [ ]:
# random agent random impatience results
random_results

In [ ]:
# arithmetic agent random impatience result
arithmetic_results

In [ ]:
mean_list = [i/10.0 for i in range(11)]
all_results = []

# params that are needed to initialize seller
value = 10
range_min = 11
range_max = 30
rounds = 100000
init_offer=range_max

# random agent
random_results = []
seller = RandomSeller(value=value, init_offer=init_offer)
for mean in mean_list:
  result = normal_imp_simulation(value,
                        range_min,
                        range_max,
                        mean = mean,
                        rounds = rounds,
                        seller=seller)
  random_results.append(result)

# simple heuristic agent, consistently performs better than random
arithmetic_results = []
n = int((init_offer-value)/2)
seller = DescentArithmeticSeller(value=value, init_offer=init_offer, init_descent = n)
for mean in mean_list:
  result = normal_imp_simulation(value,
                        range_min,
                        range_max,
                        mean = mean,
                        rounds = rounds,
                        seller=seller)
  arithmetic_results.append(result)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(mean_list, random_results, marker='o', label=f'RandomSeller')
plt.plot(mean_list, arithmetic_results, marker='o', label=f'DescentArithmeticSeller')
plt.title('Payoff vs. Imp_incr')
plt.xlabel('Mean Impatience Incremenet')
plt.ylabel('Average Episode Payoff')
plt.grid()

plt.legend()  # This will create a legend using the labels specified in the plot functions

plt.show()

# Two sellers

In [30]:
from sellers import *
from buyer import Buyer
from episode import Episode

In [31]:
import random

class EducatedBuyer(object):
    def __init__(self, max_price, imp_incr):
        self.max_price = max_price
        self.imp = 0
        self.imp_incr = imp_incr

    def check_all_offers(self, offers):
        decisions = [] # saves decision for each offer
        accept_indices = [] # saves indices of accepted offers
        accept_values = [] # saves values of accepted offers

        random_number = random.uniform(0, 1)
        if random_number <= self.imp:
            for i in range(len(offers)):
                decisions.append("walk away")
        else:
            for i, offer in enumerate(offers):
                if offer <= self.max_price:
                    decisions.append("accept offer")
                    accept_indices.append(i)
                    accept_values.append(offer)
                else:
                    decisions.append("reject but continue")

        # if more than one offer is accepted
        # NEED OT CHANGE REJECT BUT CONTINUE TO COMPETITOR WON OUT IF THERE ARE ANY ACCEPTS AT ALL
        if len(accept_indices) > 0:
            min_value = min(accept_values)
            min_indices = [i for i, value in zip(accept_indices, accept_values) if value == min_value]

            if len(min_indices) > 1:
                # randomly choose an offer to accept if there are multiple offers with the minimum value
                accept_index = random.choice(min_indices)
            else:
                # if there is a unique minimum, accept it
                accept_index = min_indices[0]
            for idx in range(len(decisions)):
              if idx == accept_index:
                decisions[idx] = "accept offer"
              else:
                decisions[idx] = "competitor won"

        self.imp += self.imp_incr
        return decisions


In [32]:
edubuyer = EducatedBuyer(15, 0.2)
edubuyer.check_all_offers([11,11])

['competitor won', 'accept offer']

In [34]:
edubuyer = EducatedBuyer(15, 0.1)
edubuyer.check_all_offers([20,20])

value = 10
range_min = 11
range_max = 30

n = int((range_max-value)/2)

random_seller = RandomSeller(value=value, init_offer=range_max)
heuristic_seller = DescentArithmeticSeller(value=value, init_offer=init_offer, init_descent = n)
sellers = [random_seller, heuristic_seller]

def two_seller_negotiation(edubuyer, sellers):
  # assume 2 sellers
  terminate = False
  while not terminate:
    offers = [seller.make_offer() for seller in sellers]
    decisions = edubuyer.check_all_offers(offers)
    terminate = ("walk away" in decisions or "accept offer" in decisions)
    # this kind of a junky way to code it 
    # but it works for now
    for i, seller in enumerate(sellers):
      new_state = {"last-offer":offers[i],
                  "comp-last-offer":(offers[:i] + offers[i+1:])[0],
                  "offers-made":seller.state["offers-made"]+1}
      seller.state = new_state
  # get reward for the agents if terminated
  profits = [None for i in range(len(sellers))]
  for i, seller in enumerate(sellers):
    if decisions[i] == "accept offer":
      profits[i]=seller.state["last-offer"]-seller.value
    elif decisions[i] == "walk away":
      profits[i]=0
    elif decisions[i] == "competitor won":
      profits[i]=0


NameError: name 'init_offer' is not defined

In [35]:
import numpy as np
import matplotlib.pyplot as plt
def show_histogram(mean, stdev):
  num_samples = 1000
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  plt.hist(samples, bins=20, density=True)
  plt.show()

def sample_one_point(mean, stdev):
  num_samples = 1
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  return samples[0]

In [36]:
# buyer distrubution
mean = 15
stdev = 3
#show_histogram(mean, stdev)
value = 10
rounds = 1000
imp_incr = 0.1

init_offer = mean+3
n = int((range_max-value)/2)
random_seller = RandomSeller(value=value, init_offer=range_max)
heuristic_seller = DescentArithmeticSeller(value=value, init_offer=init_offer, init_descent = n)
sellers = [random_seller, heuristic_seller]

total_profits = []
for i in range(rounds):
  maxprice = sample_one_point(mean,stdev)
  edubuyer = EducatedBuyer(maxprice, imp_incr)
  profits = two_seller_negotiation(edubuyer, sellers)

NameError: name 'two_seller_negotiation' is not defined

In [24]:
edubuyer = EducatedBuyer(15, 0.1)
edubuyer.check_all_offers([20,20])

value = 10
range_min = 11
range_max = 30





two_seller_negotiation(edubuyer, sellers)

NameError: name 'two_seller_negotiation' is not defined

# Tit For Tat

In [75]:
%load_ext autoreload
%autoreload 2
from sellers import *
from buyer import Buyer
from episode import Episode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import random

class EducatedBuyer(object):
    def __init__(self, max_price, imp_incr):
        self.max_price = max_price
        self.imp = 0
        self.imp_incr = imp_incr

    def check_all_offers(self, offers):
        decisions = [] # saves decision for each offer
        accept_indices = [] # saves indices of accepted offers
        accept_values = [] # saves values of accepted offers

        random_number = random.uniform(0, 1)
        if random_number <= self.imp:
            for i in range(len(offers)):
                decisions.append("walk away")
        else:
            for i, offer in enumerate(offers):
                if offer <= self.max_price:
                    decisions.append("accept offer")
                    accept_indices.append(i)
                    accept_values.append(offer)
                else:
                    decisions.append("reject but continue")

        # if more than one offer is accepted
        # NEED OT CHANGE REJECT BUT CONTINUE TO COMPETITOR WON OUT IF THERE ARE ANY ACCEPTS AT ALL
        if len(accept_indices) > 0:
            min_value = min(accept_values)
            min_indices = [i for i, value in zip(accept_indices, accept_values) if value == min_value]

            if len(min_indices) > 1:
                # randomly choose an offer to accept if there are multiple offers with the minimum value
                accept_index = random.choice(min_indices)
            else:
                # if there is a unique minimum, accept it
                accept_index = min_indices[0]
            for idx in range(len(decisions)):
              if idx == accept_index:
                decisions[idx] = "accept offer"
              else:
                decisions[idx] = "competitor won"

        self.imp += self.imp_incr
        return decisions

In [77]:
value = 10
range_min = 11
range_max = 30

#n = int((range_max-value)/2)

random_seller = RandomSeller(value=value, init_offer=range_max)
tit_for_tat_seller = TitForTatSeller(value=value, init_offer=range_max)


In [78]:
def tit_for_tat_two_seller_negotiation(edubuyer, opponent, tit_for_tat_seller):
  # assume 2 sellers
  sellers = [opponent, tit_for_tat_seller]
  terminate = False
  last_opponent_offer = None
  while not terminate:
    offers = [opponent.make_offer(), tit_for_tat_seller.make_offer(last_opponent_offer)]
    print(f'offers: {offers}')
    decisions = edubuyer.check_all_offers(offers)
    last_opponent_offer = offers[0]
    terminate = ("walk away" in decisions or "accept offer" in decisions)
    # this kind of a junky way to code it 
    # but it works for now
    for i, seller in enumerate(sellers):
      new_state = {"last-offer":offers[i],
                  "comp-last-offer":(offers[:i] + offers[i+1:])[0],
                  "offers-made":seller.state["offers-made"]+1}
      seller.state = new_state
  # get reward for the agents if terminated
  profits = [None for i in range(len(sellers))]
  for i, seller in enumerate(sellers):
    if decisions[i] == "accept offer":
      profits[i]=seller.state["last-offer"]-seller.value
    elif decisions[i] == "walk away":
      profits[i]=0
    elif decisions[i] == "competitor won":
      profits[i]=0

  return profits

In [79]:
import numpy as np
import matplotlib.pyplot as plt
def show_histogram(mean, stdev):
  num_samples = 1000
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  plt.hist(samples, bins=20, density=True)
  plt.show()

def sample_one_point(mean, stdev):
  num_samples = 1
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  return samples[0]

In [188]:
# buyer distrubution
mean = 15
stdev = 3
#show_histogram(mean, stdev)
value = 10
rounds = 100
imp_incr = 0.1



tit_for_tat_seller = TitForTatSeller(value=value, init_offer=range_max)

total_profits = []
for i in range(rounds):
  opponent = RandomSeller(value=value, init_offer=range_max)
  maxprice = int(random.uniform(range_min, range_max+1)) ##sample_one_point(mean,stdev)
  edubuyer = EducatedBuyer(maxprice, imp_incr)

  profits = tit_for_tat_two_seller_negotiation(edubuyer, opponent, tit_for_tat_seller)
  print(profits)

offers: [24, 30]
[14, 0]
offers: [12, 30]
[2, 0]
offers: [26, 30]
[16, 0]
offers: [27, 30]
offers: [24, 27]
offers: [23, 24]
[0, 0]
offers: [28, 30]
offers: [27, 28]
offers: [13, 27]
offers: [11, 13]
[1, 0]
offers: [28, 30]
[18, 0]
offers: [29, 30]
offers: [19, 29]
[0, 0]
offers: [14, 30]
[4, 0]
offers: [28, 30]
[18, 0]
offers: [12, 30]
[2, 0]
offers: [15, 30]
[5, 0]
offers: [14, 30]
[4, 0]
offers: [23, 30]
offers: [22, 23]
[0, 0]
offers: [12, 30]
[2, 0]
offers: [14, 30]
[4, 0]
offers: [22, 30]
[12, 0]
offers: [12, 30]
[2, 0]
offers: [20, 30]
offers: [16, 20]
[6, 0]
offers: [18, 30]
[8, 0]
offers: [21, 30]
[11, 0]
offers: [27, 30]
[17, 0]
offers: [18, 30]
[8, 0]
offers: [13, 30]
[3, 0]
offers: [17, 30]
offers: [14, 17]
offers: [12, 14]
offers: [11, 12]
[0, 0]
offers: [15, 30]
[5, 0]
offers: [14, 30]
[4, 0]
offers: [27, 30]
offers: [15, 27]
[5, 0]
offers: [19, 30]
offers: [17, 19]
offers: [16, 17]
offers: [15, 16]
[0, 0]
offers: [21, 30]
[11, 0]
offers: [19, 30]
offers: [13, 19]
offers:

In [42]:

edubuyer = EducatedBuyer(15, 0.1)
edubuyer.check_all_offers([20,20])

value = 10
range_min = 11
range_max = 30


tit_for_tat_two_seller_negotiation(edubuyer, random_seller, tit_for_tat_seller)

[1, 0]

# Fixed Jump Tit for Tat

In [109]:
%load_ext autoreload
%autoreload 2
from sellers import *
from buyer import Buyer
from episode import Episode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
import random

class EducatedBuyer(object):
    def __init__(self, max_price, imp_incr):
        self.max_price = max_price
        self.imp = 0
        self.imp_incr = imp_incr

    def check_all_offers(self, offers):
        decisions = [] # saves decision for each offer
        accept_indices = [] # saves indices of accepted offers
        accept_values = [] # saves values of accepted offers

        random_number = random.uniform(0, 1)
        if random_number <= self.imp:
            for i in range(len(offers)):
                decisions.append("walk away")
        else:
            for i, offer in enumerate(offers):
                if offer <= self.max_price:
                    decisions.append("accept offer")
                    accept_indices.append(i)
                    accept_values.append(offer)
                else:
                    decisions.append("reject but continue")

        # if more than one offer is accepted
        # NEED OT CHANGE REJECT BUT CONTINUE TO COMPETITOR WON OUT IF THERE ARE ANY ACCEPTS AT ALL
        if len(accept_indices) > 0:
            min_value = min(accept_values)
            min_indices = [i for i, value in zip(accept_indices, accept_values) if value == min_value]

            if len(min_indices) > 1:
                # randomly choose an offer to accept if there are multiple offers with the minimum value
                accept_index = random.choice(min_indices)
            else:
                # if there is a unique minimum, accept it
                accept_index = min_indices[0]
            for idx in range(len(decisions)):
              if idx == accept_index:
                decisions[idx] = "accept offer"
              else:
                decisions[idx] = "competitor won"

        self.imp += self.imp_incr
        return decisions

In [111]:
value = 10
range_min = 11
range_max = 30

#n = int((range_max-value)/2)

random_seller = RandomSeller(value=value, init_offer=range_max)
fixed_jump_tit_for_tat_seller = FixedJumpTitForTatSeller(value=value, init_offer=range_max)

In [112]:
def fixed_jump_tit_for_tat_two_seller_negotiation(edubuyer, opponent, fixed_jump_tit_for_tat_seller):
  # assume 2 sellers
  sellers = [opponent, fixed_jump_tit_for_tat_seller]
  terminate = False
  opponent_first_jump = 0
  while not terminate:
    offers = [opponent.make_offer(), fixed_jump_tit_for_tat_seller.make_offer(opponent_first_jump)]
    print(f'offers: {offers}')
    if opponent_first_jump == 0:
      opponent_first_jump = range_max - offers[0]
    decisions = edubuyer.check_all_offers(offers)
    terminate = ("walk away" in decisions or "accept offer" in decisions)
    # this kind of a junky way to code it 
    # but it works for now
    for i, seller in enumerate(sellers):
      new_state = {"last-offer":offers[i],
                  "comp-last-offer":(offers[:i] + offers[i+1:])[0],
                  "offers-made":seller.state["offers-made"]+1}
      seller.state = new_state
  # get reward for the agents if terminated
  profits = [None for i in range(len(sellers))]
  for i, seller in enumerate(sellers):
    if decisions[i] == "accept offer":
      profits[i]=seller.state["last-offer"]-seller.value
    elif decisions[i] == "walk away":
      profits[i]=0
    elif decisions[i] == "competitor won":
      profits[i]=0

  return profits

In [113]:
import numpy as np
import matplotlib.pyplot as plt
def show_histogram(mean, stdev):
  num_samples = 1000
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  plt.hist(samples, bins=20, density=True)
  plt.show()

def sample_one_point(mean, stdev):
  num_samples = 1
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  return samples[0]

In [181]:
# buyer distrubution
mean = 15
stdev = 3
#show_histogram(mean, stdev)
value = 10
rounds = 100
imp_incr = 0.1



total_profits = []
for i in range(rounds):
  opponent = RandomSeller(value=value, init_offer=range_max)
  fixed_jump_tit_for_tat_seller = FixedJumpTitForTatSeller(value=value, init_offer=range_max)
  maxprice = int(random.uniform(range_min, range_max+1)) ##sample_one_point(mean,stdev)
  edubuyer = EducatedBuyer(maxprice, imp_incr)

  profits = fixed_jump_tit_for_tat_two_seller_negotiation(edubuyer, opponent, fixed_jump_tit_for_tat_seller)
  print(profits)

offers: [18, 30]
[8, 0]
offers: [19, 30]
offers: [12, 19]
[0, 0]
offers: [29, 30]
offers: [24, 29]
offers: [16, 28]
offers: [13, 27]
[3, 0]
offers: [21, 30]
offers: [11, 21]
[1, 0]
offers: [23, 30]
offers: [22, 23]
[0, 0]
offers: [23, 30]
[13, 0]
offers: [15, 30]
offers: [12, 15]
[2, 0]
offers: [26, 30]
[16, 0]
offers: [15, 30]
[5, 0]
offers: [12, 30]
[2, 0]
offers: [15, 30]
offers: [12, 15]
[0, 0]
offers: [27, 30]
[17, 0]
offers: [25, 30]
offers: [22, 25]
[12, 0]
offers: [14, 30]
[4, 0]
offers: [21, 30]
offers: [16, 21]
[0, 0]
offers: [18, 30]
[8, 0]
offers: [23, 30]
offers: [20, 23]
[10, 0]
offers: [19, 30]
[9, 0]
offers: [25, 30]
[15, 0]
offers: [27, 30]
offers: [12, 27]
[2, 0]
offers: [22, 30]
offers: [21, 22]
offers: [11, 14]
[1, 0]
offers: [24, 30]
[14, 0]
offers: [17, 30]
[7, 0]
offers: [22, 30]
[12, 0]
offers: [17, 30]
[7, 0]
offers: [27, 30]
offers: [26, 27]
offers: [21, 24]
[0, 0]
offers: [11, 30]
[1, 0]
offers: [20, 30]
[10, 0]
offers: [20, 30]
[10, 0]
offers: [13, 30]
[3, 0

# (Heuristic) Competitive Seller

Competitive seller makes the first bid at 29. For following bids, it adds 1 to the descend of its opponent and adds that to its opponent's last offer. E.g. Opponent decreased bid from 30 to 28, so competitive seller's next bid is 28-(30-28+1)=25.

In [213]:
%load_ext autoreload
%autoreload 2
from sellers import *
from buyer import Buyer
from episode import Episode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
import random

class EducatedBuyer(object):
    def __init__(self, max_price, imp_incr):
        self.max_price = max_price
        self.imp = 0
        self.imp_incr = imp_incr

    def check_all_offers(self, offers):
        decisions = [] # saves decision for each offer
        accept_indices = [] # saves indices of accepted offers
        accept_values = [] # saves values of accepted offers

        random_number = random.uniform(0, 1)
        if random_number <= self.imp:
            for i in range(len(offers)):
                decisions.append("walk away")
        else:
            for i, offer in enumerate(offers):
                if offer <= self.max_price:
                    decisions.append("accept offer")
                    accept_indices.append(i)
                    accept_values.append(offer)
                else:
                    decisions.append("reject but continue")

        # if more than one offer is accepted
        # NEED OT CHANGE REJECT BUT CONTINUE TO COMPETITOR WON OUT IF THERE ARE ANY ACCEPTS AT ALL
        if len(accept_indices) > 0:
            min_value = min(accept_values)
            min_indices = [i for i, value in zip(accept_indices, accept_values) if value == min_value]

            if len(min_indices) > 1:
                # randomly choose an offer to accept if there are multiple offers with the minimum value
                accept_index = random.choice(min_indices)
            else:
                # if there is a unique minimum, accept it
                accept_index = min_indices[0]
            for idx in range(len(decisions)):
              if idx == accept_index:
                decisions[idx] = "accept offer"
              else:
                decisions[idx] = "competitor won"

        self.imp += self.imp_incr
        return decisions

In [207]:
value = 10
range_min = 11
range_max = 30

#n = int((range_max-value)/2)

random_seller = RandomSeller(value=value, init_offer=range_max)
competitive_seller = CompetitiveSeller(value=value, init_offer=range_max)


In [220]:
def competitive_two_seller_negotiation(edubuyer, opponent, competitive_seller):
  # assume 2 sellers
  sellers = [opponent, competitive_seller]
  terminate = False
  last_opponent_offer = range_max
  last_opponent_decrease = 0
  while not terminate:
    offers = [opponent.make_offer(), competitive_seller.make_offer(last_opponent_offer, last_opponent_decrease)]
    print(f'offers: {offers}')
    decisions = edubuyer.check_all_offers(offers)
    last_opponent_offer = offers[0]
    last_opponent_decrease = opponent.state["last-offer"] - last_opponent_offer
    terminate = ("walk away" in decisions or "accept offer" in decisions)
    # this kind of a junky way to code it 
    # but it works for now
    for i, seller in enumerate(sellers):
      new_state = {"last-offer":offers[i],
                  "comp-last-offer":(offers[:i] + offers[i+1:])[0],
                  "offers-made":seller.state["offers-made"]+1}
      seller.state = new_state
  # get reward for the agents if terminated
  profits = [None for i in range(len(sellers))]
  for i, seller in enumerate(sellers):
    if decisions[i] == "accept offer":
      profits[i]=seller.state["last-offer"]-seller.value
    elif decisions[i] == "walk away":
      profits[i]=0
    elif decisions[i] == "competitor won":
      profits[i]=0

  return profits

In [209]:
import numpy as np
import matplotlib.pyplot as plt
def show_histogram(mean, stdev):
  num_samples = 1000
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  plt.hist(samples, bins=20, density=True)
  plt.show()

def sample_one_point(mean, stdev):
  num_samples = 1
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  return samples[0]

In [240]:
# buyer distrubution
mean = 15
stdev = 3
#show_histogram(mean, stdev)
value = 10
rounds = 100
imp_incr = 0.1





total_profits = []
for i in range(rounds):
  opponent = RandomSeller(value=value, init_offer=range_max)
  competitive_seller = CompetitiveSeller(value=value, init_offer=range_max)
  maxprice = int(random.uniform(range_min, range_max+1)) ##sample_one_point(mean,stdev)
  edubuyer = EducatedBuyer(maxprice, imp_incr)

  profits = competitive_two_seller_negotiation(edubuyer, opponent, competitive_seller)
  print(profits)

offers: [15, 29]
[5, 0]
offers: [24, 29]
offers: [16, 17]
offers: [13, 10]
[0, 0]
offers: [17, 29]
offers: [11, 10]
[0, 0]
offers: [25, 29]
offers: [11, 19]
[1, 0]
offers: [27, 29]
offers: [15, 23]
[5, 0]
offers: [21, 29]
[11, 0]
offers: [21, 29]
offers: [12, 11]
[0, 1]
offers: [29, 29]
offers: [14, 27]
offers: [13, 10]
[0, 0]
offers: [16, 29]
[6, 0]
offers: [21, 29]
offers: [19, 11]
[0, 1]
offers: [17, 29]
[7, 0]
offers: [13, 29]
[3, 0]
offers: [21, 29]
[11, 0]
offers: [19, 29]
[9, 0]
offers: [20, 29]
[10, 0]
offers: [25, 29]
offers: [23, 19]
[0, 9]
offers: [26, 29]
offers: [14, 21]
[4, 0]
offers: [12, 29]
[2, 0]
offers: [20, 29]
offers: [18, 10]
[0, 0]
offers: [23, 29]
offers: [12, 15]
[2, 0]
offers: [16, 29]
[6, 0]
offers: [21, 29]
[11, 0]
offers: [14, 29]
[4, 0]
offers: [20, 29]
[10, 0]
offers: [23, 29]
offers: [12, 15]
[2, 0]
offers: [11, 29]
[1, 0]
offers: [17, 29]
[7, 0]
offers: [26, 29]
offers: [18, 21]
[8, 0]
offers: [24, 29]
offers: [20, 17]
[0, 0]
offers: [20, 29]
[10, 0]
of

# Reactive Seller

Reactive seller behaves similarly to competitive seller, but if its opponent drops by 5 or more, it offers 10 on the next turn.

In [234]:
%load_ext autoreload
%autoreload 2
from sellers import *
from buyer import Buyer
from episode import Episode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [235]:
import random

class EducatedBuyer(object):
    def __init__(self, max_price, imp_incr):
        self.max_price = max_price
        self.imp = 0
        self.imp_incr = imp_incr

    def check_all_offers(self, offers):
        decisions = [] # saves decision for each offer
        accept_indices = [] # saves indices of accepted offers
        accept_values = [] # saves values of accepted offers

        random_number = random.uniform(0, 1)
        if random_number <= self.imp:
            for i in range(len(offers)):
                decisions.append("walk away")
        else:
            for i, offer in enumerate(offers):
                if offer <= self.max_price:
                    decisions.append("accept offer")
                    accept_indices.append(i)
                    accept_values.append(offer)
                else:
                    decisions.append("reject but continue")

        # if more than one offer is accepted
        # NEED OT CHANGE REJECT BUT CONTINUE TO COMPETITOR WON OUT IF THERE ARE ANY ACCEPTS AT ALL
        if len(accept_indices) > 0:
            min_value = min(accept_values)
            min_indices = [i for i, value in zip(accept_indices, accept_values) if value == min_value]

            if len(min_indices) > 1:
                # randomly choose an offer to accept if there are multiple offers with the minimum value
                accept_index = random.choice(min_indices)
            else:
                # if there is a unique minimum, accept it
                accept_index = min_indices[0]
            for idx in range(len(decisions)):
              if idx == accept_index:
                decisions[idx] = "accept offer"
              else:
                decisions[idx] = "competitor won"

        self.imp += self.imp_incr
        return decisions

In [236]:
value = 10
range_min = 11
range_max = 30

#n = int((range_max-value)/2)

random_seller = RandomSeller(value=value, init_offer=range_max)
reactive_seller = ReactiveSeller(value=value, init_offer=range_max)


In [237]:
def reactive_two_seller_negotiation(edubuyer, opponent, reactive_seller):
  # assume 2 sellers
  sellers = [opponent, reactive_seller]
  terminate = False
  last_opponent_offer = range_max
  last_opponent_decrease = 0
  while not terminate:
    offers = [opponent.make_offer(), reactive_seller.make_offer(last_opponent_offer, last_opponent_decrease)]
    print(f'offers: {offers}')
    decisions = edubuyer.check_all_offers(offers)
    last_opponent_offer = offers[0]
    last_opponent_decrease = opponent.state["last-offer"] - last_opponent_offer
    terminate = ("walk away" in decisions or "accept offer" in decisions)
    # this kind of a junky way to code it 
    # but it works for now
    for i, seller in enumerate(sellers):
      new_state = {"last-offer":offers[i],
                  "comp-last-offer":(offers[:i] + offers[i+1:])[0],
                  "offers-made":seller.state["offers-made"]+1}
      seller.state = new_state
  # get reward for the agents if terminated
  profits = [None for i in range(len(sellers))]
  for i, seller in enumerate(sellers):
    if decisions[i] == "accept offer":
      profits[i]=seller.state["last-offer"]-seller.value
    elif decisions[i] == "walk away":
      profits[i]=0
    elif decisions[i] == "competitor won":
      profits[i]=0

  return profits

In [238]:
import numpy as np
import matplotlib.pyplot as plt
def show_histogram(mean, stdev):
  num_samples = 1000
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  plt.hist(samples, bins=20, density=True)
  plt.show()

def sample_one_point(mean, stdev):
  num_samples = 1
  samples_float = np.random.normal(mean, stdev, num_samples)
  samples = np.round(samples_float).astype(int)
  return samples[0]

In [246]:
# buyer distrubution
mean = 15
stdev = 3
#show_histogram(mean, stdev)
value = 10
rounds = 100
imp_incr = 0.1





total_profits = []
for i in range(rounds):
  opponent = RandomSeller(value=value, init_offer=range_max)
  reactive_seller = ReactiveSeller(value=value, init_offer=range_max)
  maxprice = int(random.uniform(range_min, range_max+1)) ##sample_one_point(mean,stdev)
  edubuyer = EducatedBuyer(maxprice, imp_incr)

  profits = reactive_two_seller_negotiation(edubuyer, opponent, reactive_seller)
  print(profits)

offers: [27, 29]
offers: [22, 23]
offers: [21, 10]
[0, 0]
offers: [19, 29]
[9, 0]
offers: [23, 29]
offers: [21, 10]
[0, 0]
offers: [12, 29]
offers: [11, 10]
[0, 0]
offers: [15, 29]
[5, 0]
offers: [19, 29]
[9, 0]
offers: [24, 29]
offers: [18, 10]
[0, 0]
offers: [27, 29]
offers: [22, 23]
[12, 0]
offers: [27, 29]
offers: [23, 23]
[0, 0]
offers: [13, 29]
offers: [12, 10]
[0, 0]
offers: [20, 29]
offers: [15, 10]
[0, 0]
offers: [11, 29]
[1, 0]
offers: [28, 29]
offers: [21, 25]
offers: [14, 10]
[0, 0]
offers: [26, 29]
[16, 0]
offers: [15, 29]
[5, 0]
offers: [21, 29]
[11, 0]
offers: [22, 29]
offers: [16, 10]
[0, 0]
offers: [20, 29]
[10, 0]
offers: [28, 29]
offers: [17, 25]
offers: [16, 10]
[0, 0]
offers: [12, 29]
[2, 0]
offers: [25, 29]
offers: [18, 10]
[0, 0]
offers: [29, 29]
offers: [17, 27]
[7, 0]
offers: [11, 29]
[1, 0]
offers: [23, 29]
offers: [20, 10]
[0, 0]
offers: [28, 29]
offers: [21, 25]
[11, 0]
offers: [22, 29]
offers: [17, 10]
[0, 0]
offers: [24, 29]
[14, 0]
offers: [20, 29]
offers